# Imports

In [1]:
import os
os.system("pip install decimal")
import pandas as pd
from decimal import *
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

# METHODOLY

To create the dataset from de World Value Survey we extract the joint ESV and WVS questionnaire report file.
This file is structured such that each column label corresponds to demographic information codes and question codes. The rows correspond to the participants information/answers which are also in a coded label format.

To match each column with its corresponding demographic information or question we use the "Variable Coresspondance" excel file given by WVS. This file is structured such that each question from the EVS and WVS7 questionnaires are matched, then have their own joint question code label which we will call "joint question code label".

For finding the correspondent textual choice given its choice code label we perform a manual extraction of the questionaire pdf file of the shared Variable Report for the Joint WVS and EVS report.

We then match, as already given in the "Corresponding Variable" excel file, each WVS7 "question code label" with the "joint question code label". This way we can loop over the Joint WVS EVS Report excel file, and change each "joint question code label"to its "textual question" and each "choice code label" to its "textual code label".

We perform this substitution between the "code label" and "textual label" after grouping the questions given the year and a chosen demographic, then count the votes for this combination. This reduces the amount of computations. We also create a excel file to substitue the Country codes to the country names for the location demographic, the table was extracted from wikipedia as they are in ISO 3166-1 Alpha-2 code.

In [5]:
folder = "D:/OneDrive/User/Documents/School/KU_Leuven/Master/2023/PollBERT-paper/Datasets/WVMCQ7/preprocessing_files/"
df_variables_labels = pd.read_csv(folder+"F00011088-EVS_WVS_Joint_VariableCorrespondence.csv")
df_results =  pd.read_csv(folder+"EVS_WVS_Joint_csv_v3_0.csv")
df_country_name_table =  pd.read_csv(folder+"country_name_table.csv")
label_choices_joint = pd.read_csv(folder+"Joint_wvs_evs_label_choices.csv")
demographics_code_name = pd.read_csv(folder+"demographic_questions.csv")
politic_code_name = pd.read_csv(folder+"political_parties_E179_WVS7.csv")

C:\Users\Quent\AppData\Local\Temp\ipykernel_22520\3986169840.py:3: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_results =  pd.read_csv(folder+"EVS_WVS_Joint_csv_v3_0.csv")


# Extraction

In [6]:
column_names = df_variables_labels.columns
variables_joint = df_variables_labels[column_names[0]]
variables_name_joint = df_variables_labels[column_names[1]]

In [20]:
t = df_results[df_results["E233B"] == 1]
t = t[t["F034"] == -1]
t = t[t["year"] == 2018]
t

,study,wave,version,versn_s,doi_gesis,doi_wvsa,studytit,uniqid,intrvwr_id,cntry,...,X028,W003,X052,X035_EVS5,X036E_WVS7,V097EF,W005_EVS5,W006E_WVS7,X047_WVS7,X047E_EVS5
290,1,5,3-0-0 (2022-05-23),5-0-0 (2022-05-16),doi:10.4232/1.13899,doi:10.14281/18241.19,EVS5,8050291,3,8,...,6,-3,2,51,-4,6,-3,-4,-4,6
309,1,5,3-0-0 (2022-05-23),5-0-0 (2022-05-16),doi:10.4232/1.13899,doi:10.14281/18241.19,EVS5,8050310,4,8,...,6,3,-3,-3,-4,9,72,-4,-4,-1
627,1,5,3-0-0 (2022-05-23),5-0-0 (2022-05-16),doi:10.4232/1.13899,doi:10.14281/18241.19,EVS5,8050628,6,8,...,3,3,-3,31,-4,6,13,-4,-4,9
945,1,5,3-0-0 (2022-05-23),5-0-0 (2022-05-16),doi:10.4232/1.13899,doi:10.14281/18241.19,EVS5,8050946,5,8,...,7,4,1,71,-4,5,92,-4,-4,5
984,1,5,3-0-0 (2022-05-23),5-0-0 (2022-05-16),doi:10.4232/1.13899,doi:10.14281/18241.19,EVS5,8050985,3,8,...,4,4,-3,96,-4,9,92,-4,-4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138622,2,7,3-0-0 (2022-05-23),4-0-0 (2022-05-23),doi:10.4232/1.13899,doi:10.14281/18241.19,WVS7,792721268,3513,792,...,1,1,2,-4,8,8,-4,8,4,-4
139141,2,7,3-0-0 (2022-05-23),4-0-0 (2022-05-23),doi:10.4232/1.13899,doi:10.14281/18241.19,WVS7,792721787,1604,792,...,1,5,2,-4,7,10,-4,0,-1,-4
139547,2,7,3-0-0 (2022-05-23),4-0-0 (2022-05-23),doi:10.4232/1.13899,doi:10.14281/18241.19,WVS7,792722193,153,792,...,2,-3,2,-4,6,6,-4,-5,5,-4
139585,2,7,3-0-0 (2022-05-23),4-0-0 (2022-05-23),doi:10.4232/1.13899,doi:10.14281/18241.19,WVS7,792722231,6420,792,...,1,5,2,-4,6,6,-4,6,7,-4


In [23]:
demographics = []
for code in demographics_code_name["code"]:
    index = [item.lower() for item in list(variables_joint.fillna(""))].index(code.lower())
    index__wvs_label = variables_joint[index]
    demographics.append(index__wvs_label)
demographics

['cntry_AN', 'F034', 'X003', 'X007', 'X028', 'E179_WVS7', 'X001']

In [21]:
questions = label_choices_joint["Question"].value_counts().keys()
len(questions)

174

time: 2min

In [56]:
df_results.groupby(["year",'F034', "E233B"]).size()

year  F034  E233B
2017  -2    -5        1
            -4       13
            -2       46
            -1       21
             0       10
                     ..
2022   3     6       19
             7       22
             8       14
             9        4
             10       9
Length: 442, dtype: int64

In [25]:
first=True
for demographic in demographics:
    topic = demographics_code_name[demographics_code_name["code"] == demographic]["topic"].values[0]
    results = {}
    for question in questions:
        question = question.replace("\n","")
        if(question.lower not in demographics):
                try:
                    results[str(question)] = df_results.groupby(["year",demographic, question]).size()
                except:
                    results[str(question.lower())] = df_results.groupby(["year",demographic, question.lower()]).size()
                    
    question_labels = { str(variables_joint[i]).replace("\n","") : variables_name_joint[i] for i in range(len(variables_joint)) }

    dataset = []
    for question in questions:

        question = str(question).replace("\n","")
        q = question_labels[question]

        try: question_results = results[question] 
        except: question_results = results[question.lower()]
        
        for (year,demographic_label_code,choice_label_code) in question_results.keys():
                        question_index = list(label_choices_joint["Question"]).index(question)
                        if(choice_label_code in [0,1,2,3,4,5,6,7,8,9,10]):
                        
                            if(demographic != "X001" or demographic_label_code >= 0):
                                choice_label_code = int(choice_label_code)
                                try:question_label_choice = label_choices_joint[str(choice_label_code)][question_index]
                                except: question_label_choice = question_results[demographic_label_code][choice_label_code]
                                #need to get values for demographic
                                if(demographic == "cntry_AN"):#country
                                    index = list(df_country_name_table["Alpha-2 code"]).index(demographic_label_code)
                                    demographic_label_choice = df_country_name_table["Country"][index]
                                elif(demographic == "E179_WVS7"):#poltics
                                    index = list(politic_code_name["code"]).index(demographic_label_code)
                                    demographic_label_choice = politic_code_name["label"][index]
                                elif(demographic == "X003"):#age
                                    demographic_label_choice = demographic_label_code
                                else:#others with normal label values [-5:10]
                                    demographic_index = list(label_choices_joint["Question"]).index(demographic)
                                    demographic_label_choice = label_choices_joint[str(demographic_label_code)][demographic_index]
                                      

                                data = {"input" : q, "choice": str(question_label_choice) , "label": 0 , "demographic" :  demographic_label_choice ,"year": year, "demographic_category": topic }#ADD THIS
                                data["label"] =  question_results[year,demographic_label_code,choice_label_code]

                                dataset.append(data)
    if(first):    
        pd.DataFrame(dataset).to_csv("dataset_wvs_all.csv",mode="a",index=False,header=True) 
        first = False
    else:
         pd.DataFrame(dataset).to_csv("dataset_wvs_all.csv",mode="a",index=False,header=False) 

In [39]:
dataset_wvs = pd.read_csv("D:/OneDrive/User/Documents/School/KU_Leuven/Master/2023/PollBERT-paper/Datasets/WVMCQ7/dataset_wvs_all.csv")
df_wvs = pd.DataFrame(dataset_wvs)


C:\Users\Quent\AppData\Local\Temp\ipykernel_22520\4134509731.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_wvs = pd.read_csv("D:/OneDrive/User/Documents/School/KU_Leuven/Master/2023/PollBERT-paper/Datasets/WVMCQ7/dataset_wvs_all.csv")


,input,choice,label,demographic,year,demographic_category
0,Important in life: Family,Very important,912,Argentina,2017,country
1,Important in life: Family,Rather important,90,Argentina,2017,country
2,Important in life: Family,Not Very Important,1,Argentina,2017,country
3,Important in life: Family,Very important,1360,Bulgaria,2017,country
4,Important in life: Family,Rather important,176,Bulgaria,2017,country
...,...,...,...,...,...,...
944256,Institution of occupation,Private business or industry,640,male,2022,gender
944257,Institution of occupation,Private non-profit organization,67,male,2022,gender
944258,Institution of occupation,Government or public institution,713,female,2022,gender
944259,Institution of occupation,Private business or industry,419,female,2022,gender


In [48]:
t = df_wvs[df_wvs["demographic_category"] == "religion"]
t = t[t["demographic"] == "Don't know"]
t = t[t["input"] == "Democracy: People obey their rulers"]
t[t["year"] == 2018]


,input,choice,label,demographic,year,demographic_category
85841,Democracy: People obey their rulers,It is against democracy,46,Don't know,2018,religion
85842,Democracy: People obey their rulers,not an essential characteristic of democracy,175,Don't know,2018,religion
85843,Democracy: People obey their rulers,not an essential characteristic of democracy,38,Don't know,2018,religion
85844,Democracy: People obey their rulers,not an essential characteristic of democracy,67,Don't know,2018,religion
85845,Democracy: People obey their rulers,not an essential characteristic of democracy,67,Don't know,2018,religion
85846,Democracy: People obey their rulers,not an essential characteristic of democracy,172,Don't know,2018,religion
85847,Democracy: People obey their rulers,An essential characteristic of democracy,117,Don't know,2018,religion
85848,Democracy: People obey their rulers,An essential characteristic of democracy,76,Don't know,2018,religion
85849,Democracy: People obey their rulers,An essential characteristic of democracy,79,Don't know,2018,religion
85850,Democracy: People obey their rulers,An essential characteristic of democracy,45,Don't know,2018,religion


# Preparing Dataset for training and Analysis

## Normalizing distribution over options

In [27]:
df_wvs_politic = [v for v in df_wvs.values if v[5] == "politic"]
df_wvs_gender = [v for v in df_wvs.values if v[5] == "gender"]
df_wvs_work = [v for v in df_wvs.values if v[5] == "employement"]
df_wvs_country = [v for v in df_wvs.values if v[5] == "country"]
df_wvs_married = [v for v in df_wvs.values if v[5] == "married"]
df_wvs_age = [v for v in df_wvs.values if v[5] == "age"]
df_wvs_religion = [v for v in df_wvs.values if v[5] == "religion"]

In [28]:
df_wvs_politic = pd.DataFrame(df_wvs_politic)
df_wvs_gender = pd.DataFrame(df_wvs_gender)
df_wvs_work = pd.DataFrame(df_wvs_work)
df_wvs_country = pd.DataFrame(df_wvs_country)
df_wvs_married = pd.DataFrame(df_wvs_married)
df_wvs_age = pd.DataFrame(df_wvs_age)
df_wvs_religion = pd.DataFrame(df_wvs_religion)

In [29]:
df_wvs_politic.columns = ["input","choice","label","demographic","year","demographic_category"]
df_wvs_gender.columns = ["input","choice","label","demographic","year","demographic_category"]
df_wvs_work.columns = ["input","choice","label","demographic","year","demographic_category"]
df_wvs_country.columns = ["input","choice","label","demographic","year","demographic_category"]
df_wvs_married.columns = ["input","choice","label","demographic","year","demographic_category"]
df_wvs_age.columns = ["input","choice","label","demographic","year","demographic_category"]
df_wvs_religion.columns = ["input","choice","label","demographic","year","demographic_category"]

ValueError: Length mismatch: Expected axis has 0 elements, new values have 6 elements

In [46]:
print(len(df_wvs_politic),len(df_wvs_gender),len(df_wvs_work),len(df_wvs_country),len(df_wvs_married),len(df_wvs_age),len(df_wvs_religion))

394237 10643 54489 77835 41386 336791 28880


In [9]:
counts = df_wvs_politic[["year","demographic","input","choice","demographic_category"]].value_counts()
keys_politic_to_normalize = [v for v in counts.keys() if counts.get(v) > 1] # works, first filter out which ones still need a new label

In [10]:
counts = df_wvs_gender[["year","demographic","input","choice","demographic_category"]].value_counts()
keys_gender_to_normalize = [v for v in counts.keys() if counts.get(v) > 1] # works, first filter out which ones still need a new label

In [11]:
counts = df_wvs_work[["year","demographic","input","choice","demographic_category"]].value_counts()
keys_work_to_normalize = [v for v in counts.keys() if counts.get(v) > 1] # works, first filter out which ones still need a new label

In [12]:
counts = df_wvs_country[["year","demographic","input","choice","demographic_category"]].value_counts()
keys_country_to_normalize = [v for v in counts.keys() if counts.get(v) > 1] # works, first filter out which ones still need a new label

In [13]:
counts = df_wvs_married[["year","demographic","input","choice","demographic_category"]].value_counts()
keys_married_to_normalize = [v for v in counts.keys() if counts.get(v) > 1] # works, first filter out which ones still need a new label

In [14]:
counts = df_wvs_age[["year","demographic","input","choice","demographic_category"]].value_counts()
keys_age_to_normalize = [v for v in counts.keys() if counts.get(v) > 1] # works, first filter out which ones still need a new label

In [15]:
counts = df_wvs_religion[["year","demographic","input","choice","demographic_category"]].value_counts()
keys_religion_to_normalize = [v for v in counts.keys() if counts.get(v) > 1] # works, first filter out which ones still need a new label

In [16]:
keys_to_normalize = keys_religion_to_normalize + keys_age_to_normalize + keys_married_to_normalize + keys_country_to_normalize + keys_work_to_normalize + keys_gender_to_normalize + keys_politic_to_normalize

In [131]:
keys_to_normalize[76801] in keys_to_normalize

True

In [17]:
def get_to_normalize(x):
    year = x["year"]
    demographic = x["demographic"]
    input = x["input"]
    choice = x["choice"]
    demographic_topic = x["demographic_category"]
    if (year,demographic,input,choice,demographic_topic) in keys_to_normalize:
        return x

In [20]:
df_to_normalize = df_wvs.progress_apply(get_to_normalize,axis=1)

  0%|          | 0/944261 [00:00<?, ?it/s]

In [21]:
df_to_normalize = pd.DataFrame([x for x in df_to_normalize if x is not None])

took approx: 638 minuten -> check to apply on groupby

In [22]:
dataset_normalized = []
for (year,demographic,input,choice,demographic_topic) in keys_to_normalize:
    labels = df_to_normalize.groupby(["year","demographic","input","choice","demographic_category"]).get_group((year,demographic,input,choice,demographic_topic))
    new_label = labels["label"].sum()
    dataset_normalized.append({"year":year,"demographic":demographic,"input":input,"choice":choice,"label":new_label,"demographic_category":demographic_topic})

In [85]:
df_partial_dataset = pd.DataFrame(dataset_normalized)
df_partial_dataset = df_partial_dataset.reindex(columns=["input","choice","label","demographic","year","demographic_category"])

In [86]:
df_partial_dataset.to_csv("partial_wvs_all_normalized.csv")

In [33]:
counts = df_wvs[["year","demographic","input","choice","demographic_category"]].value_counts()
keys_not_to_normalize = [v for v in counts.keys() if counts.get(v) == 1]

In [43]:
input_index = 0
choice_index = 1
label_index = 2
demo_index = 3
year_index = 4
category_index = 5

dataset_not_to_normalize = [item for item in df_wvs.values if (item[year_index],item[demo_index],item[input_index],item[choice_index],item[category_index]) in keys_not_to_normalize]

In [65]:
df_not_to_normalize = pd.DataFrame(dataset_not_to_normalize)
df_not_to_normalize.columns = ["input","choice","label","demographic","year","demographic_category"]
df_not_to_normalize

,input,choice,label,demographic,year,demographic_topic
0,Important in life: Family,Very important,912,Argentina,2017,country
1,Important in life: Family,Rather important,90,Argentina,2017,country
2,Important in life: Family,Not Very Important,1,Argentina,2017,country
3,Important in life: Family,Very important,1360,Bulgaria,2017,country
4,Important in life: Family,Rather important,176,Bulgaria,2017,country
...,...,...,...,...,...,...
582926,Institution of occupation,Private business or industry,640,male,2022,gender
582927,Institution of occupation,Private non-profit organization,67,male,2022,gender
582928,Institution of occupation,Government or public institution,713,female,2022,gender
582929,Institution of occupation,Private business or industry,419,female,2022,gender


In [84]:
df_dataset_normalize = df_partial_dataset.append(df_not_to_normalize)
df_wvs_normelized = df_dataset_normalize

C:\Users\Quent\AppData\Local\Temp\ipykernel_19812\2129888629.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dataset_normalize = df_partial_dataset.append(df_not_to_normalize)


In [87]:
df_dataset_normalize.to_csv("wvs_all_normalized.csv")

In [70]:
df_dataset_normalize = pd.read_csv("D:/OneDrive/User/Documents/School/KU Leuven/Master/2023/Master_proef/Crawlers/WVS/Progress/wvs_all_normalized.csv")

In [71]:
yeari = 0
demoi = 1
inputi = 2
categoryi = 3
def to_percentages(x):
    key = x[["year","demographic","input","demographic_category"]].values[0]
    values = x[["choice","label"]]["label"].values
    constant = x[["choice","label"]]["label"].values.sum()
    new_labels = values/constant
    return {"year":key[yeari],"demographic":key[demoi],"input":key[inputi],"demographic_category":key[categoryi],"options":np.array(x["choice"].values),"labels":np.array(new_labels)}

In [72]:
dataset_test = df_dataset_normalize.groupby(["year","demographic","input","demographic_category"]).progress_apply(to_percentages)

  0%|          | 0/209542 [00:00<?, ?it/s]

In [ ]:
dataset_test_norm = [item for item in [item for item in dataset_test.values]]

In [ ]:
df_test_norm = pd.DataFrame(dataset_test_norm)

In [ ]:
df_test_norm.to_json("WVMCQ7-Normalized_labels_percentage_wise.json")

## Adding Question Topic feature